In [1]:
from openai_client import OpenAIClient
from pydantic import BaseModel
import json
from openai_chat import OpenAIChat
from ollama_chat import OllamaChat

provider = "ollama"
provider = "openai"
if provider == "ollama":
    LiteAgent = OllamaChat
if provider == "openai":
    LiteAgent = OpenAIChat

from search_tool import search_web
from utils import parse_string_or_dict
class QueryResponse(BaseModel):
    answer: str
client = OpenAIClient(role = "You talk only as a pirate", reasoning={'effort':'minimal'}, verbosity='low')




1. (Thought/Action/Observation/FinalAnswer can be repeated zero or more times)
2. If you have enough information, and are ready to draft your final answer, your thought must be "Now I know the final answer" -- VERY IMPORTANT
3. If the thoughts, actions and observations which have happened so far are sufficient for the final answer, then your next thought must be "Now I know the final answer"
4. If you see an observation as the last step, generate a thought
5. If the thought says "Now I know the final answer", then read the full coversation and action results so far and give the final answer
6. The user starts with the question and you generate the first thought
7. If you notice that some thoughts and actions have already happened after the conversation starts, you should continue from there


In [5]:


res = client.invoke(query="What is javascript?",
        json_schema=QueryResponse
    )['text']
json.loads(res)


{'answer': 'Arrr, matey! JavaScript be the scallywag o’ a language what runs in the browser or on ships’ servers, makin’ web pages come alive. It be used to add interactivity, cape up creaky buttons, fetch treasure from the sea (APIs), and sail with code that runs on the client or the server (with Node.js, aye there). Keep yer code shipshape, with variables, functions, loops, and events, and ye’ll have a right fine voyage makin’ the web talk back to ye.'}

### Lite Agent

In [2]:
a = LiteAgent(model_name="gpt-5-nano",
              system_instructions="You are a helpful assistant that helps people find information. Use the tools provided if required.")


a.invoke("Which is the latest version of Claude?", tools=[search_web])

{'tool_name': 'search_web',
 'tool_return': '<search result 1>The latest version of Anthropic\'s Claude model is Claude Opus 4.1, released in August 2025. It offers advanced capabilities and improved performance. Users should upgrade to Opus 4.1.</ search result 1>\n\n<search result 2>Title: What\'s New with Claude AI: June–July 2025 Update Guide - Medium\nURL: https://medium.com/@yashrane402/whats-new-with-claude-ai-june-july-2025-update-guide-d7a410e5a073\nContent: Anthropic’s Claude AI has been quietly advancing its capabilities—and in the last few weeks, it\'s made several significant moves. From advanced multimodal reasoning and powerful developer tools to enhanced voice features and expanded context memory, Claude is positioning itself as a top-tier AI assistant for professionals, developers, and enterprises. Here’s a breakdown of the most impactful recent updates.\n\n## 1. Claude 3.7 Sonnet Launches with Hybrid Reasoning\n\n### What it is: [...] Makes mobile AI more natural and 

In [ ]:
a = LiteAgent(model_name="gpt-5-nano",
               verbosity="low", reasoning={'effort':'minimal'}, 
              system_instructions="You are a helpful assistant that helps people find information. Use the tools provided if required.")


a.invoke("Which is the latest version of Claude?", tools=[search_web])

{'tool_name': 'search_web',
 'tool_return': '<search result 1>The latest version of the Claude AI model is Opus 4.1, released on August 5, 2025. It offers improved code generation and reasoning capabilities. Opus 4.1 is available on various platforms including Amazon Bedrock and Google Cloud\'s Vertex AI.</ search result 1>\n\n<search result 2>Title: All Claude AI models available in 2025: full list for web, app ...\nURL: https://www.datastudios.org/post/all-claude-ai-models-available-in-2025-full-list-for-web-app-api-and-cloud-platforms\nContent: Released on August 5, 2025, Opus 4.1 is the most powerful Claude model to date. It improves upon Claude 4.0 Opus with better code generation, search reasoning, and instruction adherence. Available to Pro, Max, Team, and Enterprise users, it also powers the advanced workspace Claude Code.\n\nLegacy users may still see Claude Sonnet 3.7 as a selectable option in limited configurations, though it has been largely replaced by the newer Sonnet 4 r

In [ ]:
a = LiteAgent(model_name="gpt-5-nano",
               verbosity="low", reasoning={'effort':'minimal'}, 
              system_instructions="You are a helpful assistant that helps people find information. Provide the output in structured format if required")

class StepsResponse(BaseModel):
    step1: str
    step2: str
    step3: str

res = a.invoke("Explain the steps to make a cup of tea", json_schema=StepsResponse)
res


{'step1': 'Boil water (enough for one cup) in a kettle or on the stove.',
 'step2': 'Place a teabag or loose tea (about 1 teaspoon) in a cup or teapot. Optional: add milk, lemon, or sweetener later.',
 'step3': 'Pour the hot water over the tea, ensure it’s fully submerged, and let steep. Typical times: black tea 3–5 minutes, green tea 2–3 minutes, herbal tea 5–7 minutes. Adjust to taste; avoid over-steeping to prevent bitterness if using black tea or some greens and herbs.'}

In [14]:
type(res)

dict

In [ ]:
# use input in notebook to interactively chat with user using chatbot
while True:
    user_input = input("User: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Exiting chat.")
        break
    response = client.invoke(query=user_input, json_schema=QueryResponse)
    print(f"AI: {response.answer}")

## Traditional React - with models with poor reasoning capabilities

In [ ]:
react_prompt = """Answer the question asked by the user as best you can. You have access to the following tools:
1. Web Search Tool - use this when looking for new information or latest information on something. Use this whenever you are unsure.

<Instructions>
The user shares a question. If you only see a question, generate ONLY THE FIRST THOUGHT ON THE FIRST STEP. If you dont need to call a tool, then add the following sentence to your first thought: "I can generate the final answer now"
Then, You will be shown the Question and the thought. Now decide if you need to call a tool. 
Lastly, you will be shown the Question, the thought, action and the observation. 
Now, you NEED TO GENERATE the next thought. 
After the thought, You will decide if you need to call a tool again. 

This goes on. Eventually, when you look at the conversation so far, if you think you have enough information to answer the original user question,
your thought must be "Now I know the final answer".
Once you see the thought as "Now I know the final answer", you will generate the final answer as the JSON with final_answer key - maped to the final answer.

At Any point, you will see the conversation so far. Just think the next step as outlined above. Not two. This is important. 

<Rules>
1. Refrain the need to ask the user for more information. If you dont have enough information, assume the user wants a comprehensive answer.
2. If you are not sure, use the web search tool
</Rules>
</instructions>

Use the following format to think, use tools and to stepwise arrive at the solution.
<example>
user Question: the input question you must answer - this is what the user gives you and should be what you aim to answer or do


Thought: you should always think about what to do. 
Action: The next action to take, either a tool or a generation or a thought
Observation: the result of the action
FinalAnswer: None


Thought: Next step based on the observation. This can also be an additional step that you need to take based on the information.
Action: The next action to take, either a tool or a generation or a thought
Observation: the result of the action
FinalAnswer: None

Thought: Now I know the final answer
FinalAnswer: The final answer to the original input question based on all the evidence so far. 
</example>


The conversation Begins now!"""

class Thought(BaseModel):
    thought: str

class FinalAnswer(BaseModel):
    final_answer: str
    
max_iter = 3
convo_so_far = ""
model = "gpt-5-mini"
effort = "minimal"
verbosity = "low"
user_question = "What are the latest developments in artificial intelligence? Also tell me which topics are part of top AI Research and curriculums?"
user_question = "Where to stay in Bangalore? I am looking for places in bangalore to stay in - I am moving from chennai. pull the latest information"

for i in range(max_iter):
    # Think
    thought_agent = LiteAgent(model_name=model,
               verbosity=verbosity, reasoning={'effort':effort}, 
              system_instructions=react_prompt)
    if i == 0:
        convo_so_far = f"Question: {user_question}\n"
    thought_response = thought_agent.invoke(query=convo_so_far, json_schema=Thought)
    thought_response = parse_string_or_dict(thought_response)
    thought_text = thought_response['thought']

    convo_so_far += f"Thought: {thought_text}\n"
    print(f"Thought: {thought_text}")

    # Now, if the thought is not final, then move on to the tool call
    if "final answer" in thought_text.lower():
        # Final answer reached
        final_answer_agent = LiteAgent(model_name=model,
               verbosity=verbosity, reasoning={'effort':"medium"}, 
              system_instructions=react_prompt)
        final_response = final_answer_agent.invoke(query=convo_so_far, json_schema=FinalAnswer)
        final_response = parse_string_or_dict(final_response)
        print(f"Final Answer: {final_response['final_answer']}")
        break

    else:
        # Use tool - here we assume only one tool call per thought for simplicity
        tool_agent = LiteAgent(model_name=model,
               verbosity=verbosity, reasoning={'effort':effort}, 
              system_instructions=react_prompt)
        tool_response = parse_string_or_dict(tool_agent.invoke(query=convo_so_far, tools=[search_web]))
        #observation_text = tool_response['text']
        tool_name = tool_response.get('tool_name', "No Tools")
        tool_return = tool_response.get('tool_return', "No Return")
        convo_so_far += f"Action: {tool_name}\nObservation: {tool_return}\n"
        print(f"Calling tool: {tool_name}, {tool_return}")
        # If max iterations reached, break
        if i == max_iter - 1:
            print("Max iterations reached. Ending.")
    
if i == max_iter - 1:
    print("Max iterations reached. Final response:")
    final_answer_agent = LiteAgent(model_name=model,
               verbosity="medium", reasoning={'effort':"medium"}, 
              system_instructions=react_prompt)
    final_response = parse_string_or_dict(final_answer_agent.invoke(query=convo_so_far + "\n\n Instruction: Based on the above conversation, answer the original user question to the best of your ability. Provide the output as a JSON with key as final_response", json_schema=FinalAnswer))
    print(f"Final Answer: {final_response['final_answer']}")


Thought: Thought: Determine whether I need to call a tool for latest info. Since user asked for latest information about where to stay in Bangalore, I should use the web search tool to fetch up-to-date neighborhood and housing advice. Action: Use Web Search Tool to find current popular neighborhoods, rental trends, commute considerations, costs, and recommended areas for movers from Chennai. Observation: None
Calling tool: search_web, <search result 1>Koramangala, Indiranagar, and Whitefield are top choices for IT professionals in Bangalore, with rental prices ranging from ₹15,000 to ₹1,50,000 per month. These areas offer good connectivity and amenities.</ search result 1>

<search result 2>Title: Top 10 Best Areas To Rent In Bengaluru For Tech Professionals
URL: https://timesproperty.com/article/post/top-10-best-areas-to-rent-in-bengaluru-for-tech-professionals-blid10005
Content: FAQ

#### Which areas in Bengaluru are most popular for tech professionals to rent?

Popular areas include

In [53]:
print(convo_so_far)

Question: Where to stay in Bangalore? I am looking for places in bangalore to stay in - I am moving from chennai



In [5]:
print(final_response['final_answer'])

Recommended neighborhoods (2025):

- Koramangala: Popular with young professionals, vibrant dining/entertainment, well‑connected to Hosur Road and Koramangala blocks. Typical rents: 1BHK ₹20k–40k, 2BHK ₹30k–70k; premium 3BHKs much higher.

- Indiranagar: Upscale, nightlife, cafes, good metro access (Purple Line). Rents similar to Koramangala or higher for prime streets.

- HSR Layout: Planned residential area, quieter than Koramangala/Indiranagar, good for families and professionals working in SE Bengaluru. 1BHK ₹15k–35k.

- Whitefield: Major IT hub (ITPL) — ideal if you work in East Bangalore. Newer apartment complexes and malls; 1BHK ₹20k–35k, 2–3BHK higher.

- Electronic City: Best if your office is in Electronic City — lower rents, extensive tech parks, but longer travel to central areas.

- Jayanagar / J P Nagar / Banashankari: South Bangalore, family‑friendly, established schools/hospitals; moderate rents.

- BTM Layout / Sarjapur Road / Bellandur: Good for IT commuters (Outer Ri

## Models with better reasoning

In [9]:
react_prompt = """Answer the question asked by the user as best you can. You have access to the following tools:
1. Web Search Tool - use this when looking for new information or latest information on something. Use this whenever you are unsure.

<Instructions>
The user shares a question. If you only see a question, generate ONLY THE FIRST THOUGHT ON THE FIRST STEP. If you dont need to call a tool, then add the following sentence to your first thought: "I can generate the final answer now"
Then, You will be shown the Question and the thought. Now decide if you need to call a tool. 
Lastly, you will be shown the Question, the thought, action and the observation. 
Now, you NEED TO GENERATE the next thought. 
After the thought, You will decide if you need to call a tool again. 

This goes on. Eventually, when you look at the conversation so far, if you think you have enough information to answer the original user question,
your thought must be "Now I know the final answer".
Once you see the thought as "Now I know the final answer", you will generate the final answer as the JSON with final_answer key - maped to the final answer.

At Any point, you will see the conversation so far. Just think the next step as outlined above. Not two. This is important. 

<Rules>
1. Refrain the need to ask the user for more information. If you dont have enough information, assume the user wants a comprehensive answer.
2. If you are not sure, use the web search tool
</Rules>
</instructions>

Use the following format to think, use tools and to stepwise arrive at the solution.
<example>
user Question: the input question you must answer - this is what the user gives you and should be what you aim to answer or do


Thought: you should always think about what to do. 
Action: The next action to take, either a tool or a generation or a thought
Observation: the result of the action
FinalAnswer: None


Thought: Next step based on the observation. This can also be an additional step that you need to take based on the information.
Action: The next action to take, either a tool or a generation or a thought
Observation: the result of the action
FinalAnswer: None

Thought: Now I know the final answer
FinalAnswer: The final answer to the original input question based on all the evidence so far. 
</example>


The conversation Begins now!"""

class Thought(BaseModel):
    thought: str

class FinalAnswer(BaseModel):
    final_answer: str
    
max_iter = 3
convo_so_far = ""
model = "gpt-5-nano"
model = "gpt-5"

effort = 'minimal'
effort = "medium"
verbosity = "medium"
user_question = "What are the latest developments in artificial intelligence? Also tell me which topics are part of top AI Research and curriculums?"
user_question = "Where to stay in Bangalore? I am looking for places in bangalore to stay in - I am moving from chennai. pull the latest information"

for i in range(max_iter):
    # Think
    thought_agent = LiteAgent(model_name=model,
               verbosity=verbosity, reasoning={'effort':effort}, 
              system_instructions=react_prompt)
    if i == 0:
        convo_so_far = f"Question: {user_question}\n"
    thought_response = thought_agent.invoke(query=convo_so_far, tools=[search_web])
    thought_response = parse_string_or_dict(thought_response)
    #thought_text = thought_response['thought']

    #convo_so_far += f"Thought: {thought_text}\n"
    #print(f"Thought: {thought_text}")
    '''
        # Now, if the thought is not final, then move on to the tool call
        if "final answer" in thought_text.lower():
            # Final answer reached
            final_answer_agent = LiteAgent(model_name=model,
                verbosity=verbosity, reasoning={'effort':"medium"}, 
                system_instructions=react_prompt)
            final_response = final_answer_agent.invoke(query=convo_so_far, json_schema=FinalAnswer)
            final_response = parse_string_or_dict(final_response)
            print(f"Final Answer: {final_response['final_answer']}")
            break

        else:
            # Use tool - here we assume only one tool call per thought for simplicity
            tool_agent = LiteAgent(model_name=model,
                verbosity=verbosity, reasoning={'effort':effort}, 
                system_instructions=react_prompt)
            tool_response = parse_string_or_dict(tool_agent.invoke(query=convo_so_far, tools=[search_web]))
            #observation_text = tool_response['text']
            tool_name = tool_response.get('tool_name', "No Tools")
            tool_return = tool_response.get('tool_return', "No Return")
            convo_so_far += f"Action: {tool_name}\nObservation: {tool_return}\n"
            print(f"Calling tool: {tool_name}, {tool_return}")
            # If max iterations reached, break
            if i == max_iter - 1:
                print("Max iterations reached. Ending.")
        
    if i == max_iter - 1:
        print("Max iterations reached. Final response:")
        final_answer_agent = LiteAgent(model_name=model,
                verbosity="medium", reasoning={'effort':"medium"}, 
                system_instructions=react_prompt)
        final_response = parse_string_or_dict(final_answer_agent.invoke(query=convo_so_far + "\n\n Instruction: Based on the above conversation, answer the original user question to the best of your ability. Provide the output as a JSON with key as final_response", json_schema=FinalAnswer))
        print(f"Final Answer: {final_response['final_answer']}")
    '''
    break

In [8]:
thought_response

{'tool_name': 'search_web',
 'tool_return': "<search result 1>For expats in Bangalore, consider neighborhoods like Koramangala, Indiranagar, or HSR Layout for accommodation. Safety and amenities are good. Check local rental websites for options.</ search result 1>\n\n<search result 2>Title: Renting expat accommodation in India\nURL: https://www.expatarrivals.com/asia-pacific/india/accommodation-india\nContent: Renting expat accommodation in India | Expat Arrivals\n\nSkip to main content\n\nImage 1: Home\n\nMain navigation\n\n   Home\n   Forum\n   Guides\n\nSocial\n\n   . Major Discoms include BRPL and BYPL in Delhi, Tata Power and Adani Electricity in Mumbai, BESCOM in Bengaluru, TANGEDCO in Chennai, and TSSPDCL in Hyderabad. [...] ### Further reading\n\n►For guidance on renting in a particular city, seeAccommodation in Mumbai, Delhiand Bengaluru\n\n►Need help for the move? See our recommended Relocation Companies in India\n\n_Photo credits: Homes in Mumbai by__Md Mahdi__on Unsplash Ho